In [2]:
import collections
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from twokenize import tokenize

In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
Unnamed: 0            24783 non-null int64
count                 24783 non-null int64
hate_speech           24783 non-null int64
offensive_language    24783 non-null int64
neither               24783 non-null int64
class                 24783 non-null int64
tweet                 24783 non-null object
dtypes: int64(6), object(1)
memory usage: 1.3+ MB


In [4]:
data.head()
# count: (Integer) number of users who coded each tweet (min is 3, sometimes more users coded a tweet when judgments were determined to be unreliable,
# hate_speech_annotation: (Integer) number of users who judged the tweet to be hate speech,
# offensive_language_annotation: (Integer) number of users who judged the tweet to be offensive,
# neither_annotation: (Integer) number of users who judged the tweet to be neither offensive nor non-offensive,
# label: (Class Label) class label for majority of CF users (0: 'hate-speech', 1: 'offensive-language' or 2: 'neither'),
# tweet: (string)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [34]:
# label distribution
data.groupby('class').count()['tweet']

class
0     1430
1    19190
2     4163
Name: tweet, dtype: int64

In [4]:
pure_hate = data[(data['offensive_language'] == 0) & (data['neither'] == 0)]
pure_offensive = data[(data['hate_speech'] == 0) & (data['neither'] == 0)]
pure_neither = data[(data['offensive_language'] == 0) & (data['hate_speech'] == 0)]

In [57]:
print(len(pure_hate))
print(len(pure_offensive))
print(len(pure_neither))

263
14347
2872


In [5]:
pure_hate['token'] = pure_hate['tweet'].apply(func=tokenize)
pure_offensive['token'] = pure_offensive['tweet'].apply(func=tokenize)
pure_neither['token'] = pure_neither['tweet'].apply(func=tokenize)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [59]:
pure_hate.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,token
89,90,3,3,0,0,0,"""@CB_Baby24: @white_thunduh alsarabsss"" hes a ...","["", @CB_Baby24, :, @white_thunduh, alsarabsss,..."
110,111,3,3,0,0,0,"""@DevilGrimz: @VigxRArts you're fucking gay, b...","["", @DevilGrimz, :, @VigxRArts, you're, fuckin..."
184,186,3,3,0,0,0,"""@MarkRoundtreeJr: LMFAOOOO I HATE BLACK PEOPL...","["", @MarkRoundtreeJr, :, LMFAOOOO, I, HATE, BL..."
459,466,3,3,0,0,0,"""Hey go look at that video of the man that fou...","["", Hey, go, look, at, that, video, of, the, m..."
540,549,3,3,0,0,0,"""Our people"". Now is the time for the Aryan ra...","["", Our, people, "", ., Now, is, the, time, for..."


In [60]:
phate_tokens = {i: row['token'] for i, row in pure_hate.iterrows()}
poffensive_tokens = {i: row['token'] for i, row in pure_offensive.iterrows()}
pneither_tokens = {i: row['token'] for i, row in pure_neither.iterrows()}

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

In [62]:
def get_wf(corpus_lst):
    bow = CountVectorizer(min_df=2, stop_words='english', tokenizer=tokenize)
    cv = bow.fit_transform(corpus_lst)
    tmp_sum = cv.sum(axis=0)
    wf = [(word, tmp_sum[0, idx]) for word, idx in bow.vocabulary_.items()]
    wf = sorted(wf, key=lambda x: x[1], reverse=True)
    return wf

In [63]:
stop_tks = ['.', ',', ':', ';', '"', '&', 'rt']
wf_phate = get_wf(pure_hate['tweet'].tolist())
wf_phate = list(filter(lambda x: x[0] not in stop_tks ,wf_phate))
wf_poffensive = get_wf(pure_offensive['tweet'].tolist())
wf_poffensive = list(filter(lambda x: x[0] not in stop_tks ,wf_poffensive))
wf_pneither = get_wf(pure_neither['tweet'].tolist())
wf_pneither = list(filter(lambda x: x[0] not in stop_tks ,wf_pneither))

In [65]:
top_n = 20
# wf_phate[:top_n]
wf_poffensive[:top_n]
# wf_pneither[:top_n]

[('bitch', 6545),
 ('bitches', 2516),
 ('#128514', 2294),
 ('like', 1759),
 ('hoes', 1742),
 ('pussy', 1715),
 ('hoe', 1356),
 ('#8220', 1137),
 ("i'm", 1072),
 ('?', 1065),
 ('ass', 1017),
 ('!', 1008),
 ('fuck', 970),
 ('shit', 937),
 ("don't", 926),
 ('#8221', 902),
 ('got', 870),
 ('just', 840),
 ('...', 796),
 ('u', 775)]

In [66]:
wf_pneither[:top_n]

[('trash', 383),
 ('!', 380),
 ('?', 276),
 ('bird', 239),
 ('#8230', 233),
 ('charlie', 220),
 ('like', 198),
 ('...', 185),
 ('just', 176),
 ('yellow', 171),
 ('birds', 148),
 ('#9733', 140),
 ("i'm", 126),
 ("it's", 121),
 ('amp', 118),
 ('#8220', 116),
 ('yankees', 114),
 ('-', 103),
 ('..', 101),
 ('#128514', 99)]

In [19]:
def encode_feature(df_train, df_test, *args):
    
    vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=args[0], tokenizer=tokenize, binary=args[1], min_df=2, max_df=512, stop_words='english')

    tr_x = vectorizer.fit_transform(df_train['tweet']).toarray()
    ts_x = vectorizer.transform(df_test['tweet']).toarray()
    print(vectorizer.get_feature_names())
    print("Train: ", tr_x.shape)
    print("Test: ", ts_x.shape)
    return tr_x, ts_x

In [7]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)
train_y, test_y = train['class'].to_numpy(), test['class'].to_numpy()

In [156]:
# vectorizer will be used for feature selection later on
train_x, test_x = encode_feature(train, test, 128, True)

['!!', '!!!', '" &', '#128530', '#128530 ;', '#128553', '#128553 ;', '#128557', '#128557 ;', '#65039', '#65039 ;', '#8217', '& #128530', '& #128553', '& #128557', '& #65039', '& #8230', "'", '*', ', bitch', '-', '. "', '. &', '....', '2', ': "', ': &', ': bitch', 'aint', 'bad', 'best', 'better', 'big', 'bird', 'birds', 'bitch &', 'bitch ,', 'bitch ass', 'black', 'bout', 'boy', "can't", 'cause', 'charlie', 'come', 'cunt', 'cuz', 'da', 'damn', 'dat', 'day', 'dick', 'did', 'dont', 'dumb', 'eat', 'fag', 'faggot', 'feel', 'fuckin', 'fucking', 'game', 'getting', 'ghetto', 'girl', 'girls', 'going', 'gonna', 'good', 'gotta', 'hate', 'hit', "i'll", 'im', "it's", 'let', 'life', 'like bitch', 'lil', 'little', 'lmao', 'look', 'look like', 'mad', 'make', 'man', 'money', 'n', 'need', 'new', 'niggah', 'nigger', 'old', 'people', 'play', 'real', 'really', 'retarded', 'right', 'said', 'say', 'stop', 'stupid', 'talk', 'talking', 'tell', "that's", 'think', 'tho', 'time', 'today', 'trash .', 'twitter', 'ug

In [21]:
from sklearn import svm
from sklearn import metrics

In [118]:
N = 24783
classifier = svm.SVC(gamma=0.1, class_weight={0: N/1340, 1: N/19190, 2: N/4163}, C=1.0)
classifier.fit(train_x, train_y)
preds = classifier.predict(test_x)

In [103]:
print(metrics.classification_report(test_y, preds, digits=3, output_dict=False))

              precision    recall  f1-score   support

           0      0.135     0.327     0.191       300
           1      0.922     0.339     0.495      3852
           2      0.242     0.846     0.376       805

    accuracy                          0.420      4957
   macro avg      0.433     0.504     0.354      4957
weighted avg      0.764     0.420     0.458      4957



In [25]:
import threading
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest

In [142]:
train_x, test_x = encode_feature(train, test, 512, False)
counter = {}
mi = mutual_info_classif(train_x, train_y, n_neighbors=8, discrete_features=True, random_state=424)
train_x
# vec.get_feature_names()
# for i in mi.argsort()[::-1]:
#     counter[vec.get_feature_names()[i]] = counter.get(vec.get_feature_names()[i], 0) + 1
# print(sorted(counter.items(), key=lambda x: x[1], reverse=True))

['! "', '! &', '!!', '!!!', '" "', '" &', '" -', '" .', '" bitch', '" i\'m', '#12288', '#12288 ;', '#128064', '#128064 ;', '#128074', '#128074 ;', '#128076', '#128076 ;', '#128079', '#128079 ;', '#128109', '#128109 ;', '#128128', '#128128 ;', '#128129', '#128129 ;', '#128175', '#128175 ;', '#128515', '#128515 ;', '#128520', '#128520 ;', '#128525', '#128525 ;', '#128526', '#128526 ;', '#128527', '#128527 ;', '#128530', '#128530 ;', '#128536', '#128536 ;', '#128553', '#128553 ;', '#128557', '#128557 ;', '#128563', '#128563 ;', '#128564', '#128564 ;', '#128588', '#128588 ;', '#65039', '#65039 ;', '#8217', '#8217 ;', '#8217 ;s', '#9733', '#9733 ;', '#9995', '#9995 ;', '#9996', '#9996 ;', '#yankees', '& #12288', '& #128064', '& #128074', '& #128076', '& #128079', '& #128109', '& #128128', '& #128129', '& #128175', '& #128515', '& #128525', '& #128526', '& #128527', '& #128530', '& #128536', '& #128553', '& #128557', '& #128563', '& #128564', '& #128588', '& #65039', '& #8230', '& #9733', '&

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [144]:
features = SelectKBest(mutual_info_classif, k=128)
train_x = features.fit_transform(train_x, train_y)
# test_x = features.transform(test_x)
test_x = test_x[:, features.get_support(indices=True)]
print(train_x.shape)

(19826, 128)


In [145]:
classifier = svm.SVC(gamma=0.1, class_weight={0: N/1340, 1: N/19190, 2: N/4163}, C=1.0)
classifier.fit(train_x, train_y)
preds = classifier.predict(test_x)

In [146]:
print(metrics.classification_report(test_y, preds, digits=3, output_dict=False))

              precision    recall  f1-score   support

           0      0.280     0.500     0.359       284
           1      0.846     0.862     0.854      3895
           2      0.550     0.339     0.420       778

    accuracy                          0.760      4957
   macro avg      0.559     0.567     0.544      4957
weighted avg      0.767     0.760     0.758      4957



In [17]:
from sklearn.feature_extraction.text import TfidfTransformer 

In [23]:
# encode tf-idf as features
train_x, test_x = encode_feature(train, test, 512, False)
tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True) 
train_x = tfidf_transformer.fit_transform(train_x)
test_x = tfidf_transformer.transform(test_x)

['! "', '! &', '!!', '!!!', '" "', '" &', '" -', '" .', '" bitch', '" i\'m', '#1041204', '#1041204 ;', '#12288', '#12288 ;', '#128064', '#128064 ;', '#128074', '#128074 ;', '#128076', '#128076 ;', '#128079', '#128079 ;', '#128128', '#128128 ;', '#128129', '#128129 ;', '#128175', '#128175 ;', '#128515', '#128515 ;', '#128525', '#128525 ;', '#128526', '#128526 ;', '#128527', '#128527 ;', '#128530', '#128530 ;', '#128536', '#128536 ;', '#128553', '#128553 ;', '#128557', '#128557 ;', '#128563', '#128563 ;', '#128564', '#128564 ;', '#128588', '#128588 ;', '#65039', '#65039 ;', '#8217', '#8217 ;', '#8217 ;s', '#9733', '#9733 ;', '#9995', '#9995 ;', '#9996', '#9996 ;', '#yankees', '& #1041204', '& #12288', '& #128064', '& #128074', '& #128076', '& #128079', '& #128128', '& #128129', '& #128175', '& #128515', '& #128525', '& #128527', '& #128530', '& #128536', '& #128553', '& #128557', '& #128563', '& #128564', '& #128588', '& #65039', '& #8230', '& #9733', '& #9995', '& #9996', '& gt', '& lt'

In [158]:
classifier = svm.SVC(gamma=0.1, class_weight={0: N/1340, 1: N/19190, 2: N/4163}, C=1.0)
classifier.fit(train_x, train_y)
preds = classifier.predict(test_x)

In [159]:
print(metrics.classification_report(test_y, preds, digits=3, output_dict=False))

              precision    recall  f1-score   support

           0      0.248     0.597     0.351       320
           1      0.919     0.639     0.754      3817
           2      0.410     0.768     0.535       820

    accuracy                          0.657      4957
   macro avg      0.526     0.668     0.546      4957
weighted avg      0.792     0.657     0.692      4957



In [26]:
features = SelectKBest(mutual_info_classif, k=128)
train_x = features.fit_transform(train_x, train_y)
# test_x = features.transform(test_x)
test_x = test_x[:, features.get_support(indices=True)]
print(train_x.shape)

(19826, 128)


In [161]:
classifier = svm.SVC(gamma=0.1, class_weight={0: N/1340, 1: N/19190, 2: N/4163}, C=1.0)
classifier.fit(train_x, train_y)
preds = classifier.predict(test_x)
print(metrics.classification_report(test_y, preds, digits=3, output_dict=False))

              precision    recall  f1-score   support

           0      0.252     0.475     0.329       320
           1      0.856     0.833     0.845      3817
           2      0.629     0.489     0.550       820

    accuracy                          0.753      4957
   macro avg      0.579     0.599     0.575      4957
weighted avg      0.779     0.753     0.763      4957



In [42]:
# Draw a feature heat map showing the correlation of features
import seaborn as sns

train_x_df = pd.DataFrame(train_x)
train_x_df.head()
sns.heatmap(train_x_df.corr(), annot= True, fmt= '.2f')

In [6]:
import random
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup

seed = 424
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
def encode_fn(text_list):
    all_input_ids = []    
    for text in text_list:
        input_ids = tokenizer.encode(
                        text,                      
                        add_special_tokens = True,
                        max_length = 128,
                        pad_to_max_length = True,
                        return_tensors = 'pt'
                   )
        all_input_ids.append(input_ids)    
    all_input_ids = torch.cat(all_input_ids, dim=0)
    return all_input_ids

In [9]:
train_x = encode_fn(train['tweet'])
train_y = torch.tensor(train['class'].tolist())

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [10]:
test_x = encode_fn(test['tweet'])
test_y = torch.tensor(test['class'].tolist())

In [11]:
epochs = 2
batch_size = 32

train_dataset = TensorDataset(train_x, train_y)
test_dataset = TensorDataset(test_x, test_y)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [15]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3, output_attentions=False, output_hidden_states=False)

optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [16]:
for epoch in range(epochs):
    model.train()
    total_loss, total_val_loss = 0, 0
    total_eval_accuracy = 0
    for step, batch in enumerate(train_dataloader):
        model.zero_grad()
        loss, logits = model(batch[0], token_type_ids=None, attention_mask=(batch[0]>0), labels=batch[1])
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step() 
        scheduler.step()
        
    model.eval()
    for i, batch in enumerate(val_dataloader):
        with torch.no_grad():
            loss, logits = model(batch[0], token_type_ids=None, attention_mask=(batch[0]>0), labels=batch[1])
                
            total_val_loss += loss.item()
            
            logits = logits.detach().cpu().numpy()
            label_ids = batch[1].to('cpu').numpy()
            total_eval_accuracy += flat_accuracy(logits, label_ids)
    
    avg_train_loss = total_loss / len(train_dataloader)
    avg_val_loss = total_val_loss / len(val_dataloader)
    avg_val_accuracy = total_eval_accuracy / len(val_dataloader)
    
    print(f'Train loss     : {avg_train_loss}')
    print(f'Validation loss: {avg_val_loss}')
    print(f'Accuracy: {avg_val_accuracy:.2f}')
    print('\n')

KeyboardInterrupt: 